In [ ]:
from typing import Tuple, List
import numpy as np
import pandas as pd
import pandasql as pdsql
import matplotlib.pyplot as plt
import sys
import math


from typing import Tuple

# Assume these constants are defined as they are part of the formulas but not included in the function.
# They should be defined somewhere outside of this function in the actual code.
LAMBDA = 0.441  # governs the impact of the most recent match on a team’s ratings
PHI_1 = 0.518   # govern the impact of a home match on a team’s away ratings
PHI_2 = 0.552   # govern the impact away match on a team’s home ratings respectively

ALPHA = -2.3
BETA_1 = 0.0081
BETA_2 = 3.8815

# Assuming a Ratings class exists that can take four float arguments.
class Ratings:
    def __init__(self, defensive_home, offensive_home, defensive_away, offensive_away):
        self.defensive_home = defensive_home
        self.offensive_home = offensive_home
        self.defensive_away = defensive_away
        self.offensive_away = offensive_away

def calculate_ratings(
  home_home_defensive: float,
  home_home_offensive: float,
  home_away_defensive: float,
  home_away_offensive: float,
  away_home_defensive: float,
  away_home_offensive: float,
  away_away_defensive: float,
  away_away_offensive: float,
  shots_for: int,
  shots_against: int,
  corners_for: int,
  corners_against: int
  ) -> Tuple[Ratings, Ratings]:
    # Calculate performance metrics based on shots and corners
    Sa = shots_for + corners_for
    Sh = shots_against + corners_against

    # Update home team's home offensive rating
    new_home_home_offensive = max(home_home_offensive + LAMBDA * PHI_1 * (Sa - (home_home_offensive + home_away_defensive) / 2), 0)

    # Update home team's away offensive rating
    new_home_away_offensive = max(home_away_offensive + LAMBDA * (1 - PHI_1) * (Sa - (home_away_offensive + away_away_defensive) / 2), 0)

    # Update home team's home defensive rating
    new_home_home_defensive = max(home_home_defensive + LAMBDA * PHI_1 * (Sh - (away_home_offensive + home_home_defensive) / 2), 0)

    # Update home team's away defensive rating
    new_home_away_defensive = max(home_away_defensive + LAMBDA * (1 - PHI_1) * (Sh - (away_away_offensive + home_away_defensive) / 2), 0)

    # Update away team's away offensive rating
    new_away_away_offensive = max(away_away_offensive + LAMBDA * PHI_2 * (Sh - (away_away_offensive + home_away_defensive) / 2), 0)

    # Update away team's home offensive rating
    new_away_home_offensive = max(away_home_offensive + LAMBDA * (1 - PHI_2) * (Sh - (away_home_offensive + home_home_defensive) / 2), 0)

    # Update away team's away defensive rating
    new_away_away_defensive = max(away_away_defensive + LAMBDA * PHI_2 * (Sa - (home_home_offensive + away_away_defensive) / 2), 0)

    # Update away team's home defensive rating
    new_away_home_defensive = max(away_home_defensive + LAMBDA * (1 - PHI_2) * (Sa - (home_away_offensive + away_home_defensive) / 2), 0)

    # Return the updated ratings in two Ratings objects
    return (
        Ratings(new_home_home_defensive, new_home_home_offensive, new_home_away_defensive, new_home_away_offensive),
        Ratings(new_away_away_defensive, new_away_away_offensive, new_away_home_defensive, new_away_home_offensive)
    )

def get_ratings(home_team_name: str, away_team_name: str) -> Tuple[Ratings, Ratings]:
    # Get the ratings for the home and away teams
    home_home_defensive = df_teams[df_teams["TEAM"] == home_team_name]["RATING_H_DEF"].values[0]
    home_home_offensive = df_teams[df_teams["TEAM"] == home_team_name]["RATING_H_OFF"].values[0]
    home_away_defensive = df_teams[df_teams["TEAM"] == home_team_name]["RATING_A_DEF"].values[0]
    home_away_offensive = df_teams[df_teams["TEAM"] == home_team_name]["RATING_A_OFF"].values[0]

    away_away_defensive = df_teams[df_teams["TEAM"] == away_team_name]["RATING_A_DEF"].values[0]
    away_away_offensive = df_teams[df_teams["TEAM"] == away_team_name]["RATING_A_OFF"].values[0]
    away_home_defensive = df_teams[df_teams["TEAM"] == away_team_name]["RATING_H_DEF"].values[0]
    away_home_offensive = df_teams[df_teams["TEAM"] == away_team_name]["RATING_H_OFF"].values[0]

    return (
        Ratings(home_home_defensive, home_home_offensive, home_away_defensive, home_away_offensive),
        Ratings(away_away_defensive, away_away_offensive, away_home_defensive, away_home_offensive)
    )

def get_probabilities(home_team_rating: Ratings, away_team_rating: Ratings, bookie_implied_odds: float) -> Tuple[float, float]:
    K = ALPHA + BETA_1 * (home_team_rating.offensive_home + home_team_rating.defensive_home + away_team_rating.offensive_away + away_team_rating.defensive_away) + BETA_2 * bookie_implied_odds
    p_over = np.exp(K) / (1 + np.exp(K))
    p_under = 1 - p_over
    return (p_over, p_under)


In [ ]:
LIGA = "bundesliga"
BOOKIE_ODDS_OVER_COL = "B365>2.5"
BOOKIE_ODDS_UNDER_COL = "B365<2.5"

df_matches = pd.read_csv("Datasets/" + LIGA + "/matches_with_ratings.csv")

df_matches.tail(10)

In [ ]:
df_teams = pd.read_csv("Datasets/" + LIGA + "/ratings.csv")

In [ ]:
INITIAL_BANKROLL = 100
BANKROLL_HISTORY = [INITIAL_BANKROLL]
MATCHES_TO_SIMULATE = 1000
BET_SIZE = 5

df_sample = df_matches.sample(320)
sum_ev = 0

for index, row in df_sample.iterrows():
    ev_over = row["P>2.5"] * (row[BOOKIE_ODDS_OVER_COL] - 1) - (1 - row["P>2.5"])
    ev_under = row["P<2.5"] * (row[BOOKIE_ODDS_UNDER_COL] - 1) - (1 - row["P<2.5"])
    max_ev = max(ev_over, ev_under)

    if ev_over > ev_under:
        sum_ev += ev_over
        df_sample.loc[index, "BET"] = "OVER"
        df_sample.loc[index, "WON"] = (row["FTHG"] + row["FTAG"]) > 2
        if (row["FTHG"] + row["FTAG"]) > 2:
            BANKROLL_HISTORY.append(BANKROLL_HISTORY[-1] + (row[BOOKIE_ODDS_OVER_COL] - 1) * BET_SIZE)
        else:
            BANKROLL_HISTORY.append(BANKROLL_HISTORY[-1] - BET_SIZE)
    elif ev_under > ev_over:
        sum_ev += ev_under
        df_sample.loc[index, "BET"] = "UNDER"
        df_sample.loc[index, "WON"] = (row["FTHG"] + row["FTAG"]) <= 2
        if (row["FTHG"] + row["FTAG"]) <= 2:
            BANKROLL_HISTORY.append(BANKROLL_HISTORY[-1] + (row[BOOKIE_ODDS_UNDER_COL] - 1) * BET_SIZE)

        else:
            BANKROLL_HISTORY.append(BANKROLL_HISTORY[-1] - BET_SIZE)

print("EV: " + str(sum_ev * BET_SIZE))
print("final bankroll: " + str(BANKROLL_HISTORY[-1]))

In [ ]:
# Visualize bankroll history elegantly
plt.plot(BANKROLL_HISTORY)
plt.xlabel("Number of bets")
plt.ylabel("Bankroll")
plt.title("Bankroll history")
plt.show()

In [ ]:
# Plot bets won vs lost in a pie chart
won = df_sample[df_sample["WON"] == True].shape[0]
lost = df_sample[df_sample["WON"] == False].shape[0]
plt.pie([won, lost], labels=["Won", "Lost"], autopct='%1.1f%%')
plt.title("Bets won vs lost")
plt.show()

In [ ]:
import ipywidgets as widgets
from IPython.display import display

home_team = widgets.Dropdown(
    options=df_sample["HomeTeam"].unique(),
    description='Home team:',
    disabled=False,
)

away_team = widgets.Dropdown(
    options=df_sample["AwayTeam"].unique(),
    description='Away team:',
    disabled=False,
)

odds_over = widgets.FloatText(
    value=1.0,
    description='Odds O 2.5:',
    disabled=False
)

odds_under = widgets.FloatText(
    value=1.0,
    description='Odds U 2.5:',
    disabled=False
)

display(home_team, away_team, odds_over, odds_under)



def on_button_clicked(b):
    home_team_ratings, away_team_ratings = get_ratings(home_team.value, away_team.value)
    p_over, p_under = get_probabilities(home_team_ratings, away_team_ratings, 1 / odds_over.value)
    print("P(over): " + str(p_over))
    print("P(under): " + str(p_under))
    print("EV(over): " + str(p_over * odds_over.value - (1 - p_over)))
    print("EV(under): " + str(p_under * odds_under.value - (1 - p_under)))


button = widgets.Button(description="Check bet")
display(button)
button.on_click(on_button_clicked)
